Lambda School Data Science

*Unit 4, Sprint 2, Module 3*

---

# Neural Networks & GPUs (Prepare)
*aka Hyperparameter Tuning*

*aka Big Servers for Big Problems*

## Learning Objectives
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch (Optional)

# Hyperparameter Options (Learn)
<a id="p1"></a>

## Overview

Hyperparameter tuning is much more important with neural networks than it has been with any other models that we have considered up to this point. Other supervised learning models might have a couple of parameters, but neural networks can have dozens. These can substantially affect the accuracy of our models and although it can be a time consuming process is a necessary step when working with neural networks.
​
Hyperparameter tuning comes with a challenge. How can we compare models specified with different hyperparameters if our model's final error metric can vary somewhat erratically? How do we avoid just getting unlucky and selecting the wrong hyperparameter? This is a problem that to a certain degree we just have to live with as we test and test again. However, we can minimize it somewhat by pairing our experiments with Cross Validation to reduce the variance of our final accuracy values.

### Load MNIST Dataset

In [ ]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Normalizing Input Data

It's not 100% necessary to normalize/scale your input data before feeding it to a neural network, the network can learn the appropriate weights to deal with data of as long as it is numerically represented,  but it is recommended as it can help **make training faster** and **reduces the chances that gradient descent might get stuck in a local optimum**.

<https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network>

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)


### Hyperparameter Tuning Approaches:

#### 1) Babysitting AKA "Grad Student Descent".

If you fiddled with any hyperparameters yesterday, this is basically what you did. This approach is 100% manual and is pretty common among researchers where finding that 1 exact specification that jumps your model to a level of accuracy never seen before is the difference between publishing and not publishing a paper. Of course the professors don't do this themselves, that's grunt work. This is also known as the fiddle with hyperparameters until you run out of time method.

#### 2) Grid Search

Grid Search is the Grad Student galaxy brain realization of: why don't I just specify all the experiments I want to run and let the computer try every possible combination of them while I go and grab lunch. This has a specific downside in that if I specify 5 hyperparameters with 5 options each then I've just created 5^5 combinations of hyperparameters to check. Which means that I have to train 3125 different versions of my model Then if I use 5-fold Cross Validation on that then my model has to run 15,525 times. This is the brute-force method of hyperparameter tuning, but it can be very profitable if done wisely. 

When using Grid Search here's what I suggest: don't use it to test combinations of different hyperparameters, only use it to test different specifications of **a single** hyperparameter. It's rare that combinations between different hyperparameters lead to big performance gains. You'll get 90-95% of the way there if you just Grid Search one parameter and take the best result, then retain that best result while you test another, and then retain the best specification from that while you train another. This at least makes the situation much more manageable and leads to pretty good results. 

#### 3) Random Search

Do Grid Search for a couple of hours and you'll say to yourself - "There's got to be a better way." Enter Random Search. For Random search you specify a hyperparameter space and it picks specifications from that randomly, tries them out, gives you the best results and says - That's going to have to be good enough, go home and spend time with your family. 

Grid Search treats every parameter as if it was equally important, but this just isn't the case, some are known to move the needle a lot more than others (we'll talk about that in a minute). Random Search allows searching to be specified along the most important parameter and experiments less along the dimensions of less important hyperparameters. The downside of Random search is that it won't find the absolute best hyperparameters, but it is much less costly to perform than Grid Search. 

#### 4) Bayesian Methods

One thing that can make more manual methods like babysitting and gridsearch effective is that as the experimenter sees results he can then make updates to his future searches taking into account the results of past specifications. If only we could hyperparameter tune our hyperparameter tuning. Well, we kind of can. Enter Bayesian Optimization. Neural Networks are like an optimization problem within an optimization problem, and Bayesian Optimization is a search strategy that tries to take into account the results of past searches in order to improve future ones. Check out the new library `keras-tuner` for easy implementations of Bayesian methods. 


## What Hyperparameters are there to test?

- batch_size
- training epochs
- optimization algorithms
- learning rate
- momentum
- activation functions
- dropout regularization
- number of neurons in the hidden layer

There are more, but these are the most important.

## Follow Along

## Batch Size

Batch size determines how many observations the model is shown before it calculates loss/error and updates the model weights via gradient descent. You're looking for a sweet spot here where you're showing it enough observations that you have enough information to updates the weights, but not such a large batch size that you don't get a lot of weight update iterations performed in a given epoch. Feed-forward Neural Networks aren't as sensitive to bach_size as other networks, but it is still an important hyperparameter to tune. Smaller batch sizes will also take longer to train. 

Traditionally, batch size is set in powers of 2 starting at 32 up to 512. Keras defaults to a batch size of 32 if you do not specify it. Yann LeCun famously Twitted: 

> Training with large minibatches is bad for your health.
More importantly, it's bad for your test error.
Friends dont let friends use minibatches larger than 32.

Check out this paper for more reference on his tweet. https://arxiv.org/abs/1804.07612. 

Check out this SO question on why batch size is typically set in powers of two: https://datascience.stackexchange.com/questions/20179/what-is-the-advantage-of-keeping-batch-size-a-power-of-2



In [ ]:
# WARNING - may take a few minutes before any output is visible
# Seems to vary - I blame the cloud

import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(units=32):
    # create model
    model = Sequential()
    model.add(Dense(units, input_dim=784, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [32,64,512],
              'epochs': [20],
              'units': [32],
              # TODO add more params
              }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/20
938/938 [==============================] - 1s 2ms/step - loss: 0.4191 - accuracy: 0.8876
Epoch 2/20
938/938 [==============================] - 1s 2ms/step - loss: 0.2164 - accuracy: 0.9387
Epoch 3/20
938/938 [==============================] - 2s 2ms/step - loss: 0.1743 - accuracy: 0.9500
Epoch 4/20
938/938 [==============================] - 1s 2ms/step - loss: 0.1482 - accuracy: 0.9579
Epoch 5/20
938/938 [==============================] - 2s 2ms/step - loss: 0.1299 - accuracy: 0.9628
Epoch 6/20
938/938 [==============================] - 2s 2ms/step - loss: 0.1156 - accuracy: 0.9665
Epoch 7/20
938/938 [==============================] - 2s 2ms/step - loss: 0.1053 - accuracy: 0.9689
Epoch 8/20
938/938 [==============================] - 2s 2ms/step - loss: 0.0961 - accuracy: 0.9715
Epoch 9/20
938/938 [==============================] - 2s 2ms/step - loss: 0.0889 - accuracy: 0.9733
Epoch 10/20
938/938 [==============================] - 2s 2ms/step - loss: 0.0826 - accuracy: 0.9751

In [ ]:
# Final output
"""
Best: 0.9622666716575623 using {'batch_size': 64, 'epochs': 20, 'units': 32}
Means: 0.9621333479881287, Stdev: 0.003537735992205275 with: {'batch_size': 32, 'epochs': 20, 'units': 32}
Means: 0.9622666716575623, Stdev: 0.0016786280448393934 with: {'batch_size': 64, 'epochs': 20, 'units': 32}
Means: 0.9549999952316284, Stdev: 0.002786879465037353 with: {'batch_size': 512, 'epochs': 20, 'units': 32}
"""

## Optimizer

Remember that there's a different optimizers [optimizers](https://keras.io/optimizers/). At some point, take some time to read up on them a little bit. "adam" usually gives the best results. The thing to know about choosing an optimizer is that different optimizers have different hyperparameters like learning rate, momentum, etc. So based on the optimizer you choose you might also have to tune the learning rate and momentum of those optimizers after that. 

## Learning Rate

Remember that the Learning Rate is a hyperparameter that is specific to your gradient-descent based optimizer selection. A learning rate that is too high will cause divergent behavior, but a Learning Rate that is too low will fail to converge, again, you're looking for the sweet spot. I would start out tuning learning rates by orders of magnitude: [.001, .01, .1, .2, .3, .5] etc. I wouldn't go above .5, but you can try it and see what the behavior is like. 

Once you have narrowed it down, make the window even smaller and try it again. If after running the above specification your model reports that .1 is the best optimizer, then you should probably try things like [.05, .08, .1, .12, .15] to try and narrow it down. 

It can also be good to tune the number of epochs in combination with the learning rate since the number of iterations that you allow the learning rate to reach the minimum can determine if you have let it run long enough to converge to the minimum. 

In [ ]:
learning_rates = [.001, .01, .1, .2, .3, .5]

## Momentum

Momentum is a hyperparameter that is more commonly associated with Stochastic Gradient Descent. SGD is a common optimizer because it's what people understand and know, but I doubt it will get you the best results, you can try hyperparameter tuning its attributes and see if you can beat the performance from adam. Momentum is a property that decides the willingness of an optimizer to overshoot the minimum. Imagine a ball rolling down one side of a bowl and then up the opposite side a little bit before settling back to the bottom. The purpose of momentum is to try and escale local minima.

## Activation Functions

We've talked about this a little bit, typically you'l want to use ReLU for hidden layers and either Sigmoid, or Softmax for output layers of binary and multi-class classification implementations respectively, but try other activation functions and see if you can get any better results with sigmoid or tanh or something. There are a lot of activation functions that we haven't really talked about. Maybe you'll get good results with them. Maybe you won't. :) <https://keras.io/activations/>

## Network Weight Initialization

You saw how big of an effect the way that we initialize our network's weights can have on our results. There are **a lot** of what are called initialization modes. I don't understand all of them, but they can have a big affect on your model's initial accuracy. Your model will get further with less epochs if you initialize it with weights that are well suited to the problem you're trying to solve.

`init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']`

## Dropout Regularization and the Weight Constraint

The Dropout Regularization value is a percentage of neurons that you want to be randomly deactivated during training. The weight constraint is a second regularization parameter that works in tandem with dropout regularization. You should tune these two values at the same time. 

Using dropout on visible vs hidden layers might have a different effect. Using dropout on hidden layers might not have any effect while using dropout on hidden layers might have a substantial effect. You don't necessarily need to turn use dropout unless you see that your model has overfitting and generalizability problems.

## Neurons in Hidden Layer 

Remember that when we only had a single perceptron our model was only able to fit to linearly separable data, but as we have added layers and nodes to those layers our network has become a powerhouse of fitting nonlinearity in data. The larger the network and the more nodes generally the stronger the network's capacity to fit nonlinear patterns in data. The more nodes and layers the longer it will take to train a network, and higher the probability of overfitting. The larger your network gets the more you'll need dropout regularization or other regularization techniques to keep it in check. 

Typically depth (more layers) is more important than width (more nodes) for neural networks. This is part of why Deep Learning is so highly touted. Certain deep learning architectures have truly been huge breakthroughs for certain machine learning tasks. 

You might borrow ideas from other network architectures. For example if I was doing image recognition and I wasn't taking cues from state of the art architectures like resnet, alexnet, googlenet, etc. Then I'm probably going to have to do a lot more experimentation on my own before I find something that works.

There are some heuristics, but I am highly skeptical of them. I think you're better off experimenting on your own and forming your own intuition for these kinds of problems. 

- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

## Challenge
You will be expected to tune several hyperparameters in today's module project. 

Other parameter searchers (including RandomSearchCV) - https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers

# Experiment Tracking Framework (Learn)
<a id="p2"></a>

## Overview

You will notice quickly that managing the results of all the experiments you are running becomes challenging. Which set of parameters did the best? Are my results today different than my results yesterday? Although we use Ipython Notebooks to work, the format is not well suited to logging experimental results. Enter experiment tracking frameworks like [Comet.ml](https://comet.ml) and [Weights and Biases](https://wandb.ai/), and TensorBoard's Hyperparameter Dashboard. 

Those tools will help you track your experiments, store the results, and the code associated with those experiments. Experimental results can also be readily visualized to see changes in performance across any metric you care about. Data is sent to the tool as each epoch is completed, so you can also see if your model is converging. Let's check out TensorBoard today. 

## Follow Along

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime

### 1. Create Experiment Configuration
We are going to experiment with: 
* Number of units in the first dense layer
* Learning Rate
* Optimizer

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

### 2. Adapt Model Function with HParams

In [ ]:
def train_test_model(hparams):
  
  model = tf.keras.Sequential(
      [tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
       tf.keras.layers.Dense(10, activation='softmax')      
  ])

  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  elif opt_name == 'sgd':
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
  else:
    raise ValueError(f'Unexpected optimizer: {opt_name}')

  model.compile(
      optimizer=opt,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(X_train, y_train, epochs=5)
  _, accuracy = model.evaluate(X_test, y_test)

  # Python convention: if a variable doesn't need a name, give it _
  # ten_ones = [1 for _ in range(10)]

  return accuracy

For each run, log an hparams summary with the hyperparameters and final accuracy. 

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

# Basically a grid search
for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                        HP_LEARNING_RATE.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_LEARNING_RATE: learning_rate,
          HP_OPTIMIZER: optimizer
      }

      run_name = f'run-{session_num}'
      print(f'--- Starting trial: {run_name}')
      print({param.name: hparams[param] for param in hparams})
      run('logs/hparams_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4452 - accuracy: 0.8763
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2522 - accuracy: 0.9279
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2180 - accuracy: 0.9377
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1961 - accuracy: 0.9437
Epoch 5/5
313/313 [==============================] - 0s 912us/step - loss: 0.1797 - accuracy: 0.9464
--- Starting trial: run-1
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'sgd'}
Epoch 1/5
1875/1875 [==============================] - 2s 968us/step - loss: 1.7578 - accuracy: 0.4788
Epoch 2/5
1875/1875 [==============================] - 2s 984us/step - loss: 1.0819 - accuracy: 0.7373
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.7878 - accuracy: 0.8137
Epoch 4/5
1

### 4. Visualize the Results

In [ ]:
!ls

logs  sample_data


In [ ]:
!ls logs

hparams_tuning	hparam_tuning


In [ ]:
%tensorboard --logdir logs/hparams_tuning

Reusing TensorBoard on port 6006 (pid 233), started 0:03:15 ago. (Use '!kill 233' to kill it.)

<IPython.core.display.Javascript object>

### Your Turn

Pick a few hyparameters that we *have not* tuned. Using the same code above, try changing a few parameters you're interested in and submitting the results to weights & biases. :) 

## Challenge

You will be expected to use Weights & Biases to try to tune your model during your module assignment today. 

# Hyperparameters with RandomSearchCV (Learn)

## Overview

Basically `GridSearchCV` takes forever. You'll want to adopt a slightly more sophiscated strategy.

Let's also take a look at an alternative with Keras-Tuner.

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=c9bda953c380daf42f651ef681727657b2431fab3ff8253181d6aae8f1b8c6e7
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d53f0b2b080f7bfb6475d9822c7adee05d8042e179a7be5c26790ef9b19a7834
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Follow Along

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

"""
This model Tunes:
- Number of Neurons in the Hidden Layer
- Learning Rate in Adam

"""

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='./keras-tuner-trial',
    project_name='helloworld')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2219 - accuracy: 0.9358 - val_loss: 0.1126 - val_accuracy: 0.9643
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0911 - accuracy: 0.9721 - val_loss: 0.0829 - val_accuracy: 0.9750
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0607 - accuracy: 0.9814 - val_loss: 0.0801 - val_accuracy: 0.9750
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0424 - accuracy: 0.9863 - val_loss: 0.0668 - val_accuracy: 0.9797
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0315 - accuracy: 0.9899 - val_loss: 0.0734 - val_accuracy: 0.9777
Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2204 - accuracy: 0.9364 - val_loss: 0.1143 - val_accuracy: 0.9642
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0906 - accuracy: 0.9728 - val_loss: 0.0783 - val_accuracy: 0.9755

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1994 - accuracy: 0.9402 - val_loss: 0.1134 - val_accuracy: 0.9644
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0813 - accuracy: 0.9750 - val_loss: 0.0749 - val_accuracy: 0.9766
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0520 - accuracy: 0.9834 - val_loss: 0.0673 - val_accuracy: 0.9788
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0381 - accuracy: 0.9876 - val_loss: 0.0698 - val_accuracy: 0.9784
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0271 - accuracy: 0.9913 - val_loss: 0.0752 - val_accuracy: 0.9777
Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2039 - accuracy: 0.9393 - val_loss: 0.1093 - val_accuracy: 0.9673
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0804 - accuracy: 0.9752 - val_loss: 0.0843 - val_accuracy:

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4640 - accuracy: 0.8844 - val_loss: 0.2541 - val_accuracy: 0.9300
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2268 - accuracy: 0.9376 - val_loss: 0.1946 - val_accuracy: 0.9476
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1748 - accuracy: 0.9510 - val_loss: 0.1560 - val_accuracy: 0.9563
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1417 - accuracy: 0.9604 - val_loss: 0.1336 - val_accuracy: 0.9622
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1184 - accuracy: 0.9672 - val_loss: 0.1167 - val_accuracy: 0.9657
Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4706 - accuracy: 0.8839 - val_loss: 0.2545 - val_accuracy: 0.9295
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2259 - accuracy: 0.9380 - val_loss: 0.1945 - val_accuracy: 0

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2634 - accuracy: 0.9243 - val_loss: 0.1404 - val_accuracy: 0.9594
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1154 - accuracy: 0.9650 - val_loss: 0.1005 - val_accuracy: 0.9710
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0791 - accuracy: 0.9767 - val_loss: 0.0883 - val_accuracy: 0.9714
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0583 - accuracy: 0.9822 - val_loss: 0.0851 - val_accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0464 - accuracy: 0.9862 - val_loss: 0.0774 - val_accuracy: 0.9747
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2560 - accuracy: 0.9268 - val_loss: 0.1384 - val_accuracy: 0.9592
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1123 - accuracy: 0.9670 - val_loss: 0.0998 - val_accuracy: 0.9699

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5368 - accuracy: 0.8686 - val_loss: 0.2839 - val_accuracy: 0.9207
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2548 - accuracy: 0.9297 - val_loss: 0.2199 - val_accuracy: 0.9384
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2028 - accuracy: 0.9438 - val_loss: 0.1818 - val_accuracy: 0.9484
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1701 - accuracy: 0.9528 - val_loss: 0.1595 - val_accuracy: 0.9550
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1466 - accuracy: 0.9598 - val_loss: 0.1402 - val_accuracy: 0.9596
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5269 - accuracy: 0.8705 - val_loss: 0.2777 - val_accuracy: 0.9250
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2526 - accuracy: 0.9309 - val_loss: 0.2166 - val_accuracy: 0.9385

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

## Challenge

Try to apply RandomSearchCV to your module project today. 

# Review
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
    - Activation Functions
    - Optimizer
    - Number of Layers
    - Number of Neurons
    - Batch Size
    - Dropout Regulaization
    - Learning Rate
    - Number of Epochs
    - and many more
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
    - Weights & Biases
    - Comet.ml
    - By Hand / GridSearch
    - TensorBoard
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch
    - Keras-Tuner
    - Advanced Techniques

# Sources

## Additional Reading
- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
- https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/
- https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
- https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/
- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/